In [ ]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 
!pip3 install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import os
# os.chdir('/content/drive/My Drive/Research_Shanto/Shanto/Packages/imbalanced-dataset-sampler-master')
# !python setup.py install

In [ ]:
import torch
import numpy as np
# import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F

In [ ]:
aa = np.zeros(((3,4,7)))

bb = tuple([each.data for each in aa])
print(bb[1])
print(aa.shape)
aa = aa[:,-1,:]
print(aa.shape)

In [ ]:
x = torch.tensor([[ [1, 2, 3], [2,4,1] ]])
# print("aa permute ",x.permute(1,0).shape )
print(x.shape)
# print(x.unsqueeze(1).shape)
x = x.squeeze(0)
print(x.shape)

In [ ]:
c = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
d = torch.tensor([[1.0, 1.0], [0.0, 1.0]])
e = torch.matmul(c, d)
print(e)
print(c.size())

In [ ]:
### Automatic differentiation with PyTorch
x = torch.ones(2, 2, requires_grad=True)

# an operation of tensor
y = x + 2 # y inherits grad_fn

# apply operations on y
z = y * y * 3
out = z.mean()

print(out)

out.backward()

print(x.grad) # d(out)/dx

### 2.2 Transforming Tensors


In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("X shape: ", x.size())

# add dimension
print(x.unsqueeze(1).size()) 

# transpose 
torch.transpose(x, 0,1)

## 3. Emotion Dataset


In [ ]:
import re
import numpy as np
import time
from collections import defaultdict
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import csv
import pandas as pd
from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import spacy

from gensim.models.phrases import Phrases, Phraser
import logging
import gensim
from gensim.models import Word2Vec
import multiprocessing
from gensim.models import FastText as fText
# from torchsampler import ImbalancedDatasetSampler

In [ ]:
### Helper functions
import pickle

def convert_to_pickle(item, directory):
    pickle.dump(item, open(directory,"wb"))


def load_from_pickle(directory):
    return pickle.load(open(directory,"rb"))

In [ ]:
# ### read data from your Google Drive
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# !google-drive-ocamlfuse ~/google-drive
# !fusermount -u ~/google-drive
# # !google-drive-ocamlfuse drive

In [ ]:
# load data
data = pd.read_csv("/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/ashik_dataset_2.csv")
# data = load_from_pickle(directory="../input/data-input/merged_training.pkl")
# data["Sentiment"] = data["Sentiment"].astype('str') 
def get_top_data(top_n = 3000):
    top_data_df_positive1 = data[data['Sentiment'] == 1].head(top_n)
    top_data_df_negative1 = data[data['Sentiment'] == -1].head(top_n)
    top_data_df_neutral = data[data['Sentiment'] == 0].head(top_n)
    top_data_df_positive2 = data[data['Sentiment'] == 2].head(top_n)
    top_data_df_negetive2 = data[data['Sentiment'] == -2].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive1, top_data_df_negative1, top_data_df_neutral,top_data_df_positive2,top_data_df_negetive2 ])
    return top_data_df_small


# Function call to get the top 3000 from each sentiment
data["Sentiment"].replace({1:1, 2:1, 0:0, -1:2, -2:2 },inplace=True)
# data = get_top_data(top_n=3000)
data = data[data.Sentiment != 0]
data["Sentiment"].replace({2:0},inplace=True)
# Reducing to 3 sentiment

print(data.dtypes)
# data["Sentiment"].replace({"0": "Neutral", "1": "Positive", "2": "Positive", "-1": "Negetive", "-2": "Negetive"},inplace=True)

data.Sentiment.value_counts().plot.bar()




In [ ]:
print(data.head(10))
data.shape

### 3.1 Preprocessing Data


Pre-Training

#### Tokenization and Sampling

In [ ]:
# retain only text that contain less that 70 tokens to avoid too much padding
# seq = 377
data["token_size"] = data["Data"].apply(lambda x: len(x))
# data = data.loc[data['token_size'] < seq].copy()


# sampling
data = data.sample(n=13779);

In [ ]:
data.shape

#### Constructing Vocabulary and Index-Word Mapping

In [ ]:
fastText_wv = Word2Vec.load("/content/drive/My Drive/Research_Shanto/pretrained/BengWord2Vec/BengWord2Vec/posts.bin")
weights = torch.FloatTensor(fastText_wv.wv.vectors)

In [ ]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for the dataset
class ConstructVocab():
    def __init__(self, sentences):
        self.sentences = sentences
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
        
    def create_index(self):
        # for s in self.sentences:
        #     # update with individual tokens
        #     self.vocab.update(s.split())
            # update with individual tokens
            
        self.vocab.update(self.sentences)
            
        # sort the vocab
        self.vocab = sorted(self.vocab)
#         print(self.vocab)
        

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            if word == self.vocab[-1]:
                self.word2idx[word] = 0
            else:
                self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [ ]:
# construct vocab and indexing
inputs = ConstructVocab(fastText_wv.wv.vocab)

# examples of what is in the vocab
print(inputs.vocab[
             :200])
print(len(inputs.word2idx))
print(len(inputs.vocab))
# inputs.word2idx = load_from_pickle('/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/Code/char_indices.pkl')
# inputs.idx2word = load_from_pickle('/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/Code/indices_char.pkl')
# weights = load_from_pickle('/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/Code/embedded.pkl')
# print(inputs.word2idx['<pad>'])          


In [ ]:
# vectorize to tensor
input_tensor = [[ inputs.word2idx[s] if s in inputs.word2idx else '<pad>' for s in es.split()]  for es in data["Data"].values.tolist()]
input_tensor[0:2]

In [ ]:
# vocab = set()
# for t in inputs.word2idx:
#   vocab.update(t)
# inputs.vocab = vocab

In [ ]:
print(inputs.vocab)

In [ ]:
print(len(inputs.idx2word))

In [ ]:
print(inputs.word2idx)
# print(inputs.word2idx[' '])

### 3.2 Converting Data into Tensors 


In [ ]:
data.Sentiment.value_counts().plot.bar()

In [ ]:
# vectorize to tensor
input_tensor = []
for es in data["Data"]:
  temp = []
  for s in es:
    try:
      temp.append(inputs.word2idx[s])
    except:
      temp.append(0)
      
  input_tensor.append(temp)
      
# input_tensor = [[inputs.word2idx[t] for t in s.plit() ] for s in es.split()  for es in data["Data"].values.tolist() ]

In [ ]:
# examples of what is in the input tensors
input_tensor[0:2]

### 3.3 Padding data


In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [ ]:
# calculate the max_length of input tensor
max_length_inp = 300
print(max_length_inp)

In [ ]:
# weights.size

In [ ]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [ ]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]

In [ ]:
print(input_tensor[:1])

### 3.4 Binarization


In [ ]:
### convert targets to one-hot encoding vectors
emotions = list(set(data.Sentiment.unique()))
num_emotions = len(emotions)
# binarizer
mlb = preprocessing.MultiLabelBinarizer()
data_labels =  [set(emos) & set(emotions) for emos in data[['Sentiment']].values]
bin_emotions = mlb.fit_transform(data_labels)
target_tensor = np.array(bin_emotions.tolist())

In [ ]:
target_tensor[0:8] 

In [ ]:
data[0:8]

In [ ]:
get_emotion = lambda t: np.argmax(t)

In [ ]:
get_emotion(target_tensor[0])

In [ ]:
# emotion_dict = {2:0, 3:1, 4:2, 1:-1, 0:-2}
emotion_dict = {1:1, 0:0}

In [ ]:
emotion_dict[get_emotion(target_tensor[0])]

In [ ]:
print(get_emotion)

### 3.5 Split data


In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.3)

# Split the validataion further to obtain a holdout dataset (for testing) -- split 50:50
input_tensor_val, input_tensor_test, target_tensor_val, target_tensor_test = train_test_split(input_tensor_val, target_tensor_val, test_size=0.5)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val), len(input_tensor_test), len(target_tensor_test)

### 3.6 Data Loader


In [ ]:
TRAIN_BUFFER_SIZE = len(input_tensor_train)
VAL_BUFFER_SIZE = len(input_tensor_val)
TEST_BUFFER_SIZE = len(input_tensor_test)
BATCH_SIZE = 32
TRAIN_N_BATCH = TRAIN_BUFFER_SIZE // BATCH_SIZE
VAL_N_BATCH = VAL_BUFFER_SIZE // BATCH_SIZE
TEST_N_BATCH = TEST_BUFFER_SIZE // BATCH_SIZE

embedding_dim = 300
units = 512
vocab_inp_size = len(inputs.word2idx)
target_size = num_emotions
print(num_emotions)

In [ ]:
from torch.utils.data import Dataset, DataLoader
# import torch.utils.data.sampler

In [ ]:
# convert the data to tensors and pass to the Dataloader 
# to create an batch iterator



class MyData(Dataset):

    def __init__(self, X, y):
        self.data = X
        self.target = y
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x, y, x_len
    
    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)
test_dataset = MyData(input_tensor_test, target_tensor_test)

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)
val_dataset = DataLoader(val_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)
test_dataset = DataLoader(test_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [ ]:
val_dataset.batch_size
print(len(train_dataset))

## 4. Model


### 4.1 Constructing the Model


In [ ]:
# def create_emb_layer(weights_matrix, non_trainable=False):
#     num_embeddings, embedding_dim = weights_matrix.shape
#     emb_layer = nn.Embedding(num_embeddings, embedding_dim)
#     # emb_layer.load_state_dict({'weight': weights_matrix})
#     # if non_trainable:
#     #     emb_layer.weight.requires_grad = False

#     return emb_layer, num_embeddings, embedding_dim

In [ ]:
class CnnTextClassifier(nn.Module):
    def __init__(self, weights, vocab_size, EMBEDDING_SIZE,NUM_FILTERS = [64,100],num_classes=2, window_sizes=(3,3)):
        super(CnnTextClassifier, self).__init__()
        # w2vmodel = gensim.models.KeyedVectors.load('/content/drive/My Drive/Research_Shanto/pretrained/test_w2v.model')
        
        # With pretrained embeddings
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(weights))
        # Without pretrained embeddings
        # self.embedding = nn.Embedding(vocab_size, EMBEDDING_SIZE)

        self.convs = nn.ModuleList([
                                   nn.Conv2d(1, NUM_FILTERS, [window_size, EMBEDDING_SIZE], padding=(window_size - 1, 0))
                                   for window_size in window_sizes
        ])

        self.fc = nn.Linear(NUM_FILTERS * len(window_sizes), num_classes)

    def forward(self, x):
        x = self.embedding(x) # [B, T, E]
        # print("x.shape :", x.shape)

        # Apply a convolution + max_pool layer for each window size
        x = torch.unsqueeze(x, 1)
        xs = []
        for conv in self.convs:
            x2 = torch.relu(conv(x))
            x2 = torch.squeeze(x2, -1)
            # print("x2.shape :",x2.shape)
            x2 = F.max_pool1d(x2, x2.size(2))
            xs.append(x2)
        x = torch.cat(xs, 2)

        # FC
        x = x.view(x.size(0), -1)
        logits = self.fc(x)

        probs = F.softmax(logits, dim = 1)
        return probs

### 4.2 Pretesting model


In [ ]:

### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
n_layers = 1
# print(vocab_inp_size, embedding_dim, (modely.wv.vocab.keys()) )
# weights=0
vocab_inp_size = (inputs.word2idx)
model = CnnTextClassifier(weights, vocab_inp_size, embedding_dim )
model.to(device)

# obtain one sample from the data iterator
it = iter(train_dataset)
x, y, x_len = next(it)


# sort the batch first to be able to use with pac_pack sequence
xs, ys, lens = sort_batch(x, y, x_len)
print("xs.size(0) :", xs.size(0))

print("Input size: ", xs.size())
# hid = model.initialize_hidden_state(BATCH_SIZE, device)
# hid= tuple([each.data for each in hid])


output, _ = model(xs.to(device), lens, device)
# print(output.size())
print(model)

## 5. Training the Model


In [ ]:
# for name, param in model.named_parameters():                
#     if(name == "embedding.weight"):
#       param.requires_grad = False

In [ ]:
### Enabling cuda
use_cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if use_cuda else "cpu")
model = EmoGRU(weights, vocab_inp_size, embedding_dim, units, BATCH_SIZE, target_size)
model.to(device)

### loss criterion and optimizer for training
criterion = nn.CrossEntropyLoss() # the same as log_softmax + NLLLoss
optimizer = torch.optim.Adam(model.parameters())

def loss_function(y, prediction):
    """ CrossEntropyLoss expects outputs and class indices as target """
    # convert from one-hot encoding to class indices
    target = torch.max(y, 1)[1]
    # print(target.shape)
    # print(prediction.shape)
    loss = criterion(prediction, target) 
    return loss   #TODO: refer the parameter of these functions as the same
    
def accuracy(target, logit):
    ''' Obtain accuracy for training round '''
    target = torch.max(target, 1)[1] # convert from one-hot encoding to class indices
    # print("Before: ", logit)
    logit = F.softmax(logit, dim = 1)
    # print("After: ", logit)
    corrects = (torch.max(logit, 1)[1].data == target).sum()
    accuracy = 100.0 * corrects / len(logit)
    return accuracy

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()
    
    ### Initialize hidden state
    # TODO: do initialization here.
    total_loss = 0
    train_accuracy, val_accuracy = 0, 0
    
    ### Training
    for (batch, (inp, targ, lens)) in enumerate(train_dataset):
        loss = 0
        predictions, _ = model(inp.permute(1 ,0).to(device), lens, device) # TODO:don't need _   
              
        loss += loss_function(targ.to(device), predictions)
        batch_loss = (loss / int(targ.shape[1]))        
        total_loss += batch_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_accuracy = accuracy(targ.to(device), predictions)
        train_accuracy += batch_accuracy
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Val. Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.cpu().detach().numpy()))
            
    ### Validating
    for (batch, (inp, targ, lens)) in enumerate(val_dataset):        
        predictions,_ = model(inp.permute(1, 0).to(device), lens, device)        
        batch_accuracy = accuracy(targ.to(device), predictions)
        val_accuracy += batch_accuracy
    
    print('Epoch {} Loss {:.4f} -- Train Acc. {:.4f} -- Val Acc. {:.4f}'.format(epoch + 1, 
                                                             total_loss / TRAIN_N_BATCH, 
                                                             train_accuracy / TRAIN_N_BATCH,
                                                             val_accuracy / VAL_N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
new = "RezaFastText_Gru"
name = new + str(max_length_inp) + '_' + str(EPOCHS) + '_' + str(data.shape[0]) + '_' + new + '_' + str(units) 

In [ ]:
fig1 = plt.gcf()
plt.plot(y_, train_acc, label='train_accuracy')
plt.plot(y_, val_acc, label='validation_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.draw()
fig1.savefig("/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/results/" + name + "_acc.png")

In [ ]:
fig2 = plt.gcf()
plt.plot(y_, mean_loss_t, label='loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.draw()
fig2.savefig("/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/results/" + name + "_loss.png")

In [ ]:
model.parameters

In [ ]:


# test_accuracy = 0
# all_predictions = []
# x_raw = []
# y_raw = []

# device = "cuda" # we don't need GPU to do testing
# model.to("cuda")

# for (batch, (inp, targ, lens)) in (train_dataset):          
#     predictions,_ = model(inp.permute(1, 0).to(device), lens, device)        
#     batch_accuracy = accuracy(targ.to(device), predictions)
#     test_accuracy += batch_accuracy
    
#     x_raw = x_raw + [x for x in inp]
#     y_raw = y_raw + [y for y in targ]
    
#     all_predictions.append(predictions)
    
# print("Test Accuracy: ", test_accuracy / TRAIN_N_BATCH)

## 6. Evaluation on the Testing Data


In [ ]:
test_accuracy = 0
all_predictions = []
x_raw = []
y_raw = []

use_cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda" if use_cuda else "cpu")
# device = "cuda" # we don't need GPU to do testing
model.to(device)
# test_ = model.initialize_hidden_state(BATCH_SIZE, device)

for (batch, (inp, targ, lens)) in enumerate(test_dataset):    
    # test_= tuple([each.data for each in test_])
    
    predictions,test_h = model(inp.permute(1 ,0).to(device),lens, device) 
    batch_accuracy = accuracy(targ.to(device), predictions)
    print(targ, predictions)
    test_accuracy += batch_accuracy
    
    x_raw = x_raw + [x for x in inp]
    y_raw = y_raw + [y for y in targ]
    
    all_predictions.append(predictions)
    
print("Test Accuracy: ", test_accuracy.cpu().detach().numpy() / TEST_N_BATCH)

### 6.1 Confusion Matrix


In [ ]:
### Class to Properly Evaluate our Models
class Evaluate():

    def va_dist(cls, prediction, target, va_df, binarizer, name='', silent=False):
        """ Computes distance between actual and prediction through cosine distance """
        va_matrix = va_df.loc[binarizer.classes_][['valence','arousal']].values
        y_va = target.dot(va_matrix)
        F_va = prediction.dot(va_matrix)

        # dist is a one row vector with size of the test data passed(emotion)
        dist = metrics.pairwise.paired_cosine_distances(y_va, F_va)
        res = stats.describe(dist)

        # print by default (if silent=False)
        if not silent:
            print('%s\tmean: %f\tvariance: %f' % (name, res.mean, res.variance))

        return {
            'distances': dist,
            'dist_stat': res
        }

    def evaluate_class(cls, predictions, target, target2=None, silent=False):
        """ Compute only the predicted class """
        p_2_annotation = dict()

        precision_recall_fscore_support = [
            (pair[0], pair[1].mean()) for pair in zip(
                ['precision', 'recall', 'f1', 'support'],
                metrics.precision_recall_fscore_support(target, predictions)
            )
        ]

        metrics.precision_recall_fscore_support(target, predictions)

        # confusion matrix
        le = LabelEncoder()
        target_le = le.fit_transform(target)
        predictions_le = le.transform(predictions)
        cm = metrics.confusion_matrix(target_le, predictions_le)

        # prediction if two annotations are given on test data
        if target2:
            p_2_annotation = pd.DataFrame(
                [(pred, pred in set([t1,t2])) for pred, t1, t2 in zip(predictions, target, target2)],
                columns=['emo','success']
            ).groupby('emo').apply(lambda emo: emo.success.sum()/ len(emo.success)).to_dict()

        if not silent:
            print("Default Classification report")
            print(metrics.classification_report(target, predictions))

            # print if target2 was provided
            if len(p_2_annotation) > 0:
                print('\nPrecision on 2 annotations:')
                for emo in p_2_annotation:
                    print("%s: %.2f" % (emo, p_2_annotation[emo]))

            # print accuracies, precision, recall, and f1
            print('\nAccuracy:')
            print(metrics.accuracy_score(target, predictions))
            print("Correct Predictions: ", metrics.accuracy_score(target, predictions,normalize=False))
            for to_print in precision_recall_fscore_support[:3]:
                print( "%s: %.2f" % to_print )

            # normalizing the values of the consfusion matrix
            print('\nconfusion matrix\n %s' % cm)
            print('(row=expected, col=predicted)')
            cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            cls.plot_confusion_matrix(cm_normalized, le.classes_, 'Confusion matrix Normalized')

        return {
            'precision_recall_fscore_support': precision_recall_fscore_support,
            'accuracy': metrics.accuracy_score(target, predictions),
            'p_2_annotation': p_2_annotation,
            'confusion_matrix': cm
        }

    def predict_class(cls, X_train, y_train, X_test, y_test,
                      pipeline, silent=False, target2=None):
        """ Predicted class,then run some performance evaluation """
        pipeline.fit(X_train, y_train)
        predictions = pipeline.predict(X_test)
        print("predictions computed....")
        return cls.evaluate_class(predictions, y_test, target2, silent)

    def evaluate_prob(cls, prediction, target_rank, target_class, binarizer, va_df, silent=False, target2=None):
        """ Evaluate through probability """
        # Run normal class evaluator
        predict_class = binarizer.classes_[prediction.argmax(axis=1)]
        class_eval = cls.evaluate_class(predict_class, target_class, target2, silent)

        if not silent:
            print('\n - First Emotion Classification Metrics -')
            print('\n - Multiple Emotion rank Metrics -')
            print('VA Cosine Distance')

        classes_dist = [
            (
                emo,
                cls.va_dist(
                    prediction[np.array(target_class) == emo],
                    target_rank[np.array(target_class) == emo],
                    va_df,
                    binarizer,
                    emo,
                    silent)
                ) for emo in binarizer.classes_
        ]
        avg_dist = cls.va_dist(prediction, target_rank, va_df, binarizer, 'avg', silent)

        coverage_error = metrics.coverage_error(target_rank, prediction)
        average_precision_score = metrics.average_precision_score(target_rank, prediction)
        label_ranking_average_precision_score = metrics.label_ranking_average_precision_score(target_rank, prediction)
        label_ranking_loss = metrics.label_ranking_loss(target_rank, prediction)

        # recall at 2
        # obtain top two predictions
        top2_pred = [set([binarizer.classes_[i[0]], binarizer.classes_[i[1]]]) for i in (prediction.argsort(axis=1).T[-2:].T)]
        recall_at_2 = pd.DataFrame(
            [
            t in p for t, p in zip(target_class, top2_pred)
            ], index=target_class, columns=['recall@2']).groupby(level=0).apply(lambda emo: emo.sum()/len(emo))

        # combine target into sets
        if target2:
            union_target = [set(t) for t in zip(target_class, target2)]
        else:
            union_target = [set(t) for t in zip(target_class)]

        # precision at k
        top_k_pred = [
            [set([binarizer.classes_[i] for i in i_list]) for i_list in (prediction.argsort(axis=1).T[-i:].T)]
            for i in range(2, len(binarizer.classes_)+1)]
        precision_at_k = [
            ('p@' + str(k+2), np.array([len(t & p)/(k+2) for t, p in zip(union_target, top_k_pred[k])]).mean())
            for k in range(len(top_k_pred))]

        # do this if silent= False
        if not silent:
            print('\n')
            print(recall_at_2)
            print('\n')
            print('p@k')
            for pk in precision_at_k:
                print(pk[0] + ':\t' + str(pk[1]))
            print('\ncoverage_error: %f' % coverage_error)
            print('average_precision_score: %f' % average_precision_score)
            print('label_ranking_average_precision_score: %f' % label_ranking_average_precision_score)
            print('label_ranking_loss: %f' % label_ranking_loss)

        return {
            'class_eval': class_eval,
            'recall_at_2': recall_at_2.to_dict(),
            'precision_at_2': precision_at_k,
            'classes_dist': classes_dist,
            'avg_dist': avg_dist,
            'coverage_error': coverage_error,
            'average_precision_score': average_precision_score,
            'label_ranking_average_precision_score': label_ranking_average_precision_score,
            'label_ranking_loss': label_ranking_loss
        }


    def predict_prob(cls, X_train, y_train, X_test, y_test, label_test, pipeline, binarizer, va_df, silent=False, target2=None):
        """ Output predcations based on training and labels """
        pipeline.fit(X_train, y_train)
        predictions = pipeline.predict_proba(X_test)
        pred_to_mlb = [np.where(pipeline.classes_ == emo)[0][0] for emo in binarizer.classes_.tolist()]
        return cls.evaluate_prob(predictions[:,pred_to_mlb], y_test, label_test, binarizer, va_df, silent, target2)


    def plot_confusion_matrix(cls, cm, my_tags, title='Confusion matrix', cmap=plt.cm.Blues):
        """ Plotting the confusion_matrix"""
        plt.rc('figure', figsize=(4, 4), dpi=100)
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(my_tags))
        target_names = my_tags
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
        
        # add normalized values inside the Confusion matrix
        fmt = '.2f'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.savefig('/content/drive/My Drive/Research_Shanto/Datasets/Ashik Bhai_Sentiment/results/' + name +'_confusion.png')

In [ ]:
evaluator = Evaluate()

final_predictions = []

for p in all_predictions:
    for sub_p in p:
        final_predictions.append(sub_p.cpu().detach().numpy())
        
predictions = [np.argmax(p).item() for p in final_predictions]
targets = [np.argmax(t).item() for t in y_raw]
correct_predictions = float(np.sum(predictions == targets))

# predictions
predictions_human_readable = ((x_raw, predictions))
# actual targets
target_human_readable = ((x_raw,  targets))

# emotion_dict = {0: '0', 1: '-1', 2: '-2', 3: '1', 4: '2'}
emotion_dict = {1:1, 0:0,2:2}
# convert results into dataframe
model_test_result = pd.DataFrame(predictions_human_readable[1],columns=["emotion"])
test = pd.DataFrame(target_human_readable[1], columns=["emotion"])

model_test_result.emotion = model_test_result.emotion.map(lambda x: emotion_dict[int(float(x))])
test.emotion = test.emotion.map(lambda x: emotion_dict[int(x)])

evaluator.evaluate_class(model_test_result.emotion, test.emotion );